<a href="https://colab.research.google.com/github/Norod/hebrew-gpt_neo/blob/main/hebrew-gpt_neo-xl/Norod78_hebrew_gpt_neo_xl_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tokenizers==0.10.2 transformers==4.6.0

     |████████████████████████████████| 3.3MB 4.1MB/s 
     |████████████████████████████████| 2.3MB 51.2MB/s 
     |████████████████████████████████| 901kB 53.8MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
  
tokenizer = AutoTokenizer.from_pretrained("Norod78/hebrew-gpt_neo-xl")
model = AutoModelForCausalLM.from_pretrained("Norod78/hebrew-gpt_neo-xl", pad_token_id=tokenizer.eos_token_id)

In [3]:
prompt_text = "אני אוהב שוקולד ועוגות"
max_len = 512
sample_output_num = 3
seed = 1000

In [4]:
import numpy as np
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = 0 if torch.cuda.is_available()==False else torch.cuda.device_count()

print(f"device: {device}, n_gpu: {n_gpu}")

np.random.seed(seed)
torch.manual_seed(seed)
if n_gpu > 0:
    torch.cuda.manual_seed_all(seed)

model.to(device)

encoded_prompt = tokenizer.encode(
    prompt_text, add_special_tokens=False, return_tensors="pt")

encoded_prompt = encoded_prompt.to(device)

if encoded_prompt.size()[-1] == 0:
        input_ids = None
else:
        input_ids = encoded_prompt

print("input_ids = " + str(input_ids))

if input_ids != None:
  max_len += len(encoded_prompt[0])
  if max_len > 2048:
    max_len = 2048

print("Updated max_len = " + str(max_len))

device: cuda, n_gpu: 1
input_ids = tensor([[ 2202, 22283, 10984,   631,  5679]], device='cuda:0')
Updated max_len = 517


In [10]:
stop_token = "<|endoftext|>"
new_lines = "\n\n\n"

sample_outputs = model.generate(
    input_ids,
    do_sample=True, 
    max_length=max_len, 
    top_k=50, 
    top_p=0.95, 
    num_return_sequences=sample_output_num
)

print(100 * '-' + "\n\t\tOutput\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):

  text = tokenizer.decode(sample_output, skip_special_tokens=True)
  
  # Remove all text after the stop token
  text = text[: text.find(stop_token) if stop_token else None]

  # Remove all text after 3 newlines
  text = text[: text.find(new_lines) if new_lines else None]

  print("\n{}: {}".format(i, text))
  print("\n" + 100 * '-')

----------------------------------------------------------------------------------------------------
		Output
----------------------------------------------------------------------------------------------------

0: אני אוהב שוקולד ועוגות גבינה, אבל עוגות שוקולד הן לא ממש כוס התה שלי, לפחות לא על פי רוב. העוגה הקבועה שלי...
זה מתכון של אמא שלי, עם הרבה מרכיבים. כשהכנתי את העוגה הזאת ידעתי שאני רוצה שהיא תהיה פרווה, אבל לא היה לי זמן...

----------------------------------------------------------------------------------------------------

1: אני אוהב שוקולד ועוגות קצפת, ואני אוהב שוקו וסופגניות" מספר לנו ג'וניור ומוסיף שהוא מעדיף להיות "ילד טוב". לאחר זמן מה, ג'וניור מבהיר לנו: "לא משנה כמה שוקולדים אכין, בסוף תמיד תמיד, תמיד, אני תמיד אוכל את השוקולד שאני הכי אוהב". ואכן, ג'וניור אוהב מאוד את כל שוקולד באשר הוא, הוא מספר לנו על שוקולד חלב ועוגיות עם נגיעות של שוקולד מריר, הוא לא אוהב שוקולד לבן, אבל מכין לנו בראוניז, עם ריבת חלב, וחלב חם, כאשר לרוב שוקולד הלבן הוא השוקולד המועדף עלינו, א